# A Tool to Identify Neighboring Water Systems

## Why?

__Aim: identify neighboring water systems.__ This could be useful for:  

* Identifying if/which violations might contaminate neighboring water systems (WS)
* Feature engineering for a predictive model (e.g. violations in previous quarter in neighboring WS)
* Serving as a source of information for private (public too?) wells owners to identify potential treats to - or assess the contamination risks of - their water.


## How?

The table WATER_SYSTEM contains a ZIP code for every water system. From this ZIP code, there are probably 2 possiblities:

1. Identify neighboring water systems by computing distance between their (centroid) coordinates. Building on the work done by Aaron [here](https://github.com/codeforboston/safe-water/blob/master/code/python/notebooks/stations_to_systems_distance/systems_to_station_distance.ipynb).

2. Identify neighboring water systems with the shapes (polygons) of the ZIP codes area and finding the neighboring polygons.

__In this document, I try the second idea.__

This could be coded manually, but maybe a free - and stable - API exists that does exactly that? ==> **I DID NOT FIND ONE SO FAR**

[EDIT, idea from Ding]: Another possibility could be to work with google map? But the addresses in SDWIS are from the "legal entity", so not necessarily useful...

## Some useful links for understanding ZIP codes

* Locate ZIP codes, with the small [app from Ben Fry](https://benfry.com/zipdecode/)
* Wikipedia has a [map of the ZIP codes zones](https://en.wikipedia.org/wiki/ZIP_Code#/media/File:ZIP_Code_zones.svg)


In [257]:
import pymysql
import pandas as pd
import numpy as np

import requests # to read the data from the REST API of EPA Envirofacts
import csv # needed as data accessed through the REST API are .csv (other possibilities are .xml or .xls)

import datetime as dt # might be needed to handle the dates in the dataset more easily

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import pgeocode # one possible way to identify neighboring ZIP codes to test. https://github.com/symerio/pgeocode
import uszipcode # another possible way to test. https://pypi.org/project/uszipcode/
# from uszipcode import Zipcode

import geopandas as gpd
from shapely.geometry import Point, Polygon

For now, I am loading the data directly from the EPA SDWIS using their API. In the future, data should be loaded from our own database.   

**I am not sure that the data extraction that I do here is fully correct, but for the purpose of developping this tool, it is sufficient.**

In [141]:
# !! takes 5min

# more on the API: https://www.epa.gov/enviro/envirofacts-data-service-api

# Not sure why, but today the API is only working when retrieving 1000 rows or less... 
# So let's loop to get all the water systems.

# notes on the API:
#     - WATER_SYSTEM = table
#     - PWS_ACTIVITY_CODE/A ==> select only active water systems
#     - ROWS/0:150000 ==> I know that they are less than 150'000 water systems...

# We first query to get the first 1000 rows, and then we will append more rows:
CSV_URL = 'http://data.epa.gov/efservice/WATER_SYSTEM/PWS_ACTIVITY_CODE/A/ROWS/0:1000/CSV'
with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    initial_WS = list(cr)        
    WATER_SYSTEM = pd.DataFrame(initial_WS)

lower_bounds = range(1000, 150000, 1000) # steps of the for loop, rows to query, starting at 1000
for step in lower_bounds:
    CSV_URL = 'http://data.epa.gov/efservice/WATER_SYSTEM/PWS_ACTIVITY_CODE/A/ROWS/' +\
    str(step) + ':' + str(step+1000) + '/CSV'
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        step_data = list(cr)
        step_WS = pd.DataFrame(step_data)
    WATER_SYSTEM = WATER_SYSTEM.append(step_WS, ignore_index=True)
    


,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,WATER_SYSTEM.PWSID,WATER_SYSTEM.PWS_NAME,WATER_SYSTEM.NPM_CANDIDATE,WATER_SYSTEM.PRIMACY_AGENCY_CODE,WATER_SYSTEM.EPA_REGION,WATER_SYSTEM.SEASON_BEGIN_DATE,WATER_SYSTEM.SEASON_END_DATE,WATER_SYSTEM.PWS_ACTIVITY_CODE,WATER_SYSTEM.PWS_DEACTIVATION_DATE,WATER_SYSTEM.PWS_TYPE_CODE,...,WATER_SYSTEM.ZIP_CODE,WATER_SYSTEM.COUNTRY_CODE,WATER_SYSTEM.STATE_CODE,WATER_SYSTEM.SOURCE_WATER_PROTECTION_CODE,WATER_SYSTEM.SOURCE_PROTECTION_BEGIN_DATE,WATER_SYSTEM.OUTSTANDING_PERFORMER,WATER_SYSTEM.OUTSTANDING_PERFORM_BEGIN_DATE,WATER_SYSTEM.CITIES_SERVED,WATER_SYSTEM.COUNTIES_SERVED,
1,MD0020017,GLEN BURNIE-BROADNECK,Y,MD,03,,,A,,CWS,...,21108,US,MD,Y,30-JUN-03,N,,Not Reported,Anne Arundel,None
2,MO5301396,MIMOSA BEACH CONDOMINIUMS,Y,MO,07,04-01,10-31,A,,TNCWS,...,65324-0000,US,MO,N,,,,Not Reported,Camden,None
3,TX0200703,OASIS RV PARK,Y,TX,06,01-01,12-31,A,,TNCWS,...,77541-7241,US,TX,,,,,Not Reported,Brazoria,None
4,TX0570040,CITY OF COPPELL,Y,TX,06,,,A,,CWS,...,75019-9478,US,TX,Y,31-AUG-06,,,Not Reported,Dallas,None
5,TX1010592,DOWDELL PUD,Y,TX,06,,,A,,CWS,...,77019-2191,US,TX,N,,,,Not Reported,Harris,None
6,TX2130042,SOMERVELL COUNTY WATER DISTRICT - WHEELE,Y,TX,06,,,A,,CWS,...,76043-1386,US,TX,N,,,,Not Reported,Somervell,None
7,PA2450896,PUB 570,Y,PA,03,01-01,12-31,A,,TNCWS,...,18302,US,PA,,,,,Not Reported,Monroe,None
8,WI6320351,LAX CO GOOSE IS ARTESIAN WELL,Y,WI,05,01-01,12-31,A,,TNCWS,...,54601,US,WI,,,,,STODDARD,La Crosse,None
9,WI6320350,LAX CO GOOSE IS PK SHELTER 4,Y,WI,05,04-15,09-30,A,,TNCWS,...,54601,US,WI,,,,,STODDARD,La Crosse,None


In [251]:
## some DATA CLEANING

WS_raw = WATER_SYSTEM.copy()

# set the first row to set is as header:
new_header = WS_raw.iloc[0] # grab the first row for the header
new_header = new_header.str.split('.').str[1] # we remove the redundant table name (WATER_SYSTEM) in column names
new_header = new_header.str.lower() # set to lower case, as less annoying
WS_raw = WS_raw[1:] # take the data less the header row
WS_raw.columns = new_header # set the header row as the df header

# we remove the last column of null, that is an artifact of the extraction:
WS_raw = WS_raw.dropna(axis = 1, how='all') # axis = 1 = columns

# remove null rows:
WS_raw = WS_raw.dropna(axis = 0, how='all') # axis = 0 = rows

# remove extraction errors for PWSID (e.g. WS_raw.iloc[1005,:]) ==> PWSID = html tag..
WS_raw = WS_raw[~WS_raw['pwsid'].str.contains('<')] # remove rows where PWSID == html tag
WS_raw = WS_raw[~WS_raw['pwsid'].str.contains('error')] # remove rows where PWSID == error
WS_raw = WS_raw[~WS_raw['pwsid'].str.contains(':')] # remove rows where PWSID == : 
WS_raw = WS_raw[~WS_raw['pwsid'].str.contains(';')] # remove rows where PWSID == ;
WS_raw = WS_raw[~WS_raw['pwsid'].str.contains('begin')] # remove rows where PWSID == ;
WS_raw = WS_raw[~WS_raw['pwsid'].str.contains('WATER_SYSTEM.PWSID')] # remove rows where PWSID == ;

# Because I want to work with ZIP codes here, 
WS_raw = WS_raw.dropna(axis = 0, subset=['zip_code']) # axis = 0 = rows


# WS_raw.isnull().sum()

In [264]:
WS_raw.head().T

,1,2,3,4,5
0,,,,,
pwsid,MD0020017,MO5301396,TX0200703,TX0570040,TX1010592
pws_name,GLEN BURNIE-BROADNECK,MIMOSA BEACH CONDOMINIUMS,OASIS RV PARK,CITY OF COPPELL,DOWDELL PUD
npm_candidate,Y,Y,Y,Y,Y
primacy_agency_code,MD,MO,TX,TX,TX
epa_region,03,07,06,06,06
season_begin_date,,04-01,01-01,,
season_end_date,,10-31,12-31,,
pws_activity_code,A,A,A,A,A
pws_deactivation_date,,,,,


In [252]:
# Cleaning ZIP codes:

# some are in this form: 77541-7241
# ==> need to keep first 5 digits only:
WS_raw.zip_code = WS_raw.zip_code.astype(str).str[0:5] # convert object to string to use string slicing

In [254]:
# END OF CLEANING PROCESS:
WS = WS_raw.copy()

### Test _pgeocode_

In [255]:
# Example, find distance between two ZIP codes:

dist = pgeocode.GeoDistance('us')
print(dist.query_postal_code(WS.zip_code[1], WS.zip_code[2])) # WHAT IS THE UNIT?

# dist.query_postal_code(["75013", "75014", "75015"], ["69006", "69005", "69004"])


1421.195891275818


In [15]:
# How to use findNearbyPostalCodes ? (https://www.geonames.org/export/ws-overview.html)

# continue the work of Aaron: [here](https://github.com/codeforboston/safe-water/blob/master/code/python/notebooks/stations_to_systems_distance/systems_to_station_distance.ipynb)

## ZIP Codes Shapefile

ZIP Code Tabulation Areas (ZCTAs) are approximate area representations of U.S. Postal Service (USPS) ZIP Code service areas that the Census Bureau creates to present statistical data for each decennial census. It can be downloaded [here](https://catalog.data.gov/dataset/tiger-line-shapefile-2015-2010-nation-u-s-2010-census-5-digit-zip-code-tabulation-area-zcta5-na).

Not sure if a problem yet (from [wikipedia](https://en.wikipedia.org/wiki/ZIP_Code#Statistics):
"As of 2015, there were over 42,000 ZIP Codes in the United States. ZIP Codes are used not only for tracking of mail but also in gathering geographical statistics in the United States. The U.S. Census Bureau calculates approximate boundaries of ZIP Codes areas, which it calls ZIP Code Tabulation Areas (ZCTAs). Statistical census data is then provided for these approximate areas. The geographic data provided for these areas includes the latitude and longitude of the center-point of the ZCTAs. There are approximately 32,000 ZCTAs. The reason that there is not one ZCTA for every ZIP Code is that PO Boxes are excluded, since only populated areas are included in the Census data. The Census Bureau provides many statistical data sets for ZIP Codes, but does not keep up-to-date datasets of all ZCTAs. Complete datasets providing a similar approximate geographic extent are commercially available."

For now, I load the shapefile locally and it is not uploaded on Github as it is too large (838MB). A better solution should be found...

In [275]:
# loading shapefile:
zip_shp = gpd.read_file('../../../../tl_2015_us_zcta510/tl_2015_us_zcta510.shp')
print(zip_shp.shape) # 33'000 ZIP codes 
zip_shp.head()
# info on the data, there p.78 (3-70)
# https://www2.census.gov/geo/pdfs/maps-data/data/tiger/tgrshp2015/TGRSHP2015_TechDoc.pdf

(33144, 10)


,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,43451,43451,B5,G6350,S,63411475,157689,+41.3183010,-083.6174935,"POLYGON ((-83.674464 41.331119, -83.6744449999..."
1,43452,43452,B5,G6350,S,121783676,13437379,+41.5157923,-082.9809454,"POLYGON ((-83.067745 41.537718, -83.067729 41...."
2,43456,43456,B5,G6350,S,9389361,999166,+41.6468445,-082.8226641,"(POLYGON ((-82.8566 41.681222, -82.856831 41.6..."
3,43457,43457,B5,G6350,S,48035540,0,+41.2673266,-083.4274645,"POLYGON ((-83.467474 41.268186, -83.4676039999..."
4,43458,43458,B5,G6350,S,2573816,39915,+41.5304461,-083.2133648,"POLYGON ((-83.222292 41.531025, -83.2222819999..."


In [295]:
# FINDING NEIGHBORS
# I will build on this: https://gis.stackexchange.com/questions/281652/find-all-neighbors-using-geopandas

# take a subset of the data to first try:
zip_shp_subset = zip_shp.head(100)
# WS_subset = WS[['pwsid', 'zip_code', 'state_code']].head(100)

zip_shp_subset.loc[:,"my_neighbors"] = None  # add NEIGHBORS column

for index, row in zip_shp_subset.iterrows(): 
    # get 'touching' ZIP
    
    neighbors = zip_shp_subset[zip_shp_subset.geometry.touches(row['geometry'])].ZCTA5CE10.tolist()
    # remove own ZIP(ZCTA5CE10) from the list. NEEDED? HOW?
#     neighbors = neighbors.remove(row.ZCTA5CE10)
#     neighbors = [ name for name in neighbors if zip_shp_subset.ZCTA5CE10 != name ]
#     print(neighbors)
    # add names of neighbors to the my_neighbors column
    zip_shp_subset.at[index, "my_neighbors"] = ", ".join(neighbors)


In [294]:
zip_shp_subset

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,my_neighbors
0,43451,43451,B5,G6350,S,63411475,157689,+41.3183010,-083.6174935,"POLYGON ((-83.674464 41.331119, -83.6744449999...","43462, 43466"
1,43452,43452,B5,G6350,S,121783676,13437379,+41.5157923,-082.9809454,"POLYGON ((-83.067745 41.537718, -83.067729 41....",
2,43456,43456,B5,G6350,S,9389361,999166,+41.6468445,-082.8226641,"(POLYGON ((-82.8566 41.681222, -82.856831 41.6...",
3,43457,43457,B5,G6350,S,48035540,0,+41.2673266,-083.4274645,"POLYGON ((-83.467474 41.268186, -83.4676039999...","43466, 43467"
4,43458,43458,B5,G6350,S,2573816,39915,+41.5304461,-083.2133648,"POLYGON ((-83.222292 41.531025, -83.2222819999...",
5,43460,43460,B5,G6350,S,7158544,791729,+41.6011920,-083.5649985,"POLYGON ((-83.576634 41.60105, -83.577534 41.6...",
6,43462,43462,B5,G6350,S,66526720,65743,+41.2837850,-083.7228646,"POLYGON ((-83.78597599999999 41.299238, -83.78...",43451
7,43463,43463,B5,G6350,S,19415,0,+41.5086650,-083.5080344,POLYGON ((-83.50844699999999 41.50923299999999...,
8,43464,43464,B5,G6350,S,86139720,12290475,+41.4048795,-082.9241092,"POLYGON ((-82.95293599999999 41.430172, -82.90...",
9,43465,43465,B5,G6350,S,28559487,101977,+41.5654721,-083.5003020,"(POLYGON ((-83.52781999999999 41.581865, -83.5...",


# TO DO

1. check if correct ==> It looks like it is correct... Tested with this: http://maps.huge.info/zip.htm
2. find a way to run it through all zip codes without spending days looping...
    * maybe by first taking the intersection btw the ZTCA and water systems ZIP? Not sure if sufficient
    * maybe do it on a subset of not too far ZIP codes first

In [305]:
# just saving the subset to test:
# zip_shp_subset.to_csv(path_or_buf='../../../../zip_shp_subset.csv')